In [13]:
import numpy as np
import pandas as pd
import random

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, Lasso
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from lime.lime_text import LimeTextExplainer

import time


## Binary Classification

Attempting GridSearchCV for hyperparameter selection. The penalty is elasticnet, and saga is the solver. We used a max iteration count of 2000 to ensure convergence due to the large dataset size.

In [16]:
num_splits = 5
cv_k = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)

param_grid = {
    'C': [1, 3, 5, 10],
    'penalty': ['elasticnet'],
    'solver': ['saga'],
    'l1_ratio': [0.1, 0.5],
    'max_iter': [2000],
}

for i in range(num_splits):
    start_time = time.time()
    train_file_path = f'./F24_Proj3_data/split_{2}/train.csv'
    test_file_path = f'./F24_Proj3_data/split_{2}/test.csv'
    test_y_file_path = f'./F24_Proj3_data/split_{2}/test_y.csv'

    # Load data
    X_train = pd.read_csv(train_file_path).iloc[:, 3:]
    y_train = pd.read_csv(train_file_path).iloc[:, 1]

    X_test = pd.read_csv(test_file_path).iloc[:, 2:]
    y_test = pd.read_csv(test_y_file_path).iloc[:, 1]

    log_reg = LogisticRegression()
    
    grid_search = GridSearchCV(estimator=log_reg, 
                               param_grid = param_grid,
                               cv=cv_k,
                               n_jobs=-1,
                               verbose=1,
                               scoring='accuracy')
    
    grid_search.fit(X_train, y_train)

    print("\nFold ", i+1) 
    # Access the best parameters and best estimator
    print("Best hyperparameters:", grid_search.best_params_)
    print("Best cross-validation score:", grid_search.best_score_)
    
    # Get the best model
    best_model = grid_search.best_estimator_
    
    # Make predictions with the best model
    y_pred = best_model.predict(X_test)

    
    # Evaluate the model on the test set
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Test set accuracy: {accuracy}")

Fitting 5 folds for each of 8 candidates, totalling 40 fits

Fold  0
Best hyperparameters: {'C': 10, 'l1_ratio': 0.1, 'max_iter': 2000, 'penalty': 'elasticnet', 'solver': 'saga'}
Best cross-validation score: 0.94784
Test set accuracy: 0.94772
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Fold  1
Best hyperparameters: {'C': 10, 'l1_ratio': 0.1, 'max_iter': 2000, 'penalty': 'elasticnet', 'solver': 'saga'}
Best cross-validation score: 0.94784
Test set accuracy: 0.94772
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Fold  2
Best hyperparameters: {'C': 10, 'l1_ratio': 0.1, 'max_iter': 2000, 'penalty': 'elasticnet', 'solver': 'saga'}
Best cross-validation score: 0.94784
Test set accuracy: 0.94772
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Fold  3
Best hyperparameters: {'C': 10, 'l1_ratio': 0.1, 'max_iter': 2000, 'penalty': 'elasticnet', 'solver': 'saga'}
Best cross-validation score: 0.94784
Test set accuracy: 0.94772
Fitting 5 folds for each of 

Now we'll train the logistic regression model using the best C and l1 ratio we found earlier. C = 10, l1 = 0.1.

In [ ]:
num_splits = 5

log_reg = LogisticRegression(
    penalty='elasticnet',
    solver='saga',
    l1_ratio=0.1,  
    C=10,
    max_iter=1000,
    n_jobs=-1,
    random_state=42
)

for i in range(num_splits):
    start_time = time.time()
    train_file_path = f'./F24_Proj3_data/split_{i+1}/train.csv'
    test_file_path = f'./F24_Proj3_data/split_{i+1}/test.csv'
    test_y_file_path = f'./F24_Proj3_data/split_{i+1}/test_y.csv'

    # Load data
    X_train = pd.read_csv(train_file_path).iloc[:, 3:]
    y_train = pd.read_csv(train_file_path).iloc[:, 1]

    X_test = pd.read_csv(test_file_path).iloc[:, 2:]
    y_test = pd.read_csv(test_y_file_path).iloc[:, 1]

    log_reg.fit(X_train, y_train)

    y_pred_proba = log_reg.predict_proba(X_test)[:, 1]

    auc_score = roc_auc_score(y_test, y_pred_proba)
    print(f'Split {i+1}: AUC Score for LogisticRegression: {auc_score:.7f}')
    print(f'| Execution time: {round(time.time() - start_time, 4)} seconds')

Split 1: AUC Score for LogisticRegression: 0.9870942
| Execution time: 29.1046 seconds
Split 2: AUC Score for LogisticRegression: 0.9867907
| Execution time: 29.5183 seconds
Split 3: AUC Score for LogisticRegression: 0.9864187
| Execution time: 29.9895 seconds
Split 4: AUC Score for LogisticRegression: 0.9869783
| Execution time: 29.8482 seconds


In [12]:
import joblib

In [ ]:
joblib.dump(model, 'logistic_model.pkl')

## Interpretability Analysis

Using split 1 and the corresponding trained model, implement an interpretability approach to identify which parts of each review have an impact on the sentiment prediction. Apply your method to 5 randomly selected positive reviews and 5 randomly selected negative reviews from the split 1 test data.

We will use the bag of words approach, which disregards word order and context, representing each review by presence or absence of words and their frequency.

In [8]:
train_file_path = f'./F24_Proj3_data/split_1/train.csv'
test_file_path = f'./F24_Proj3_data/split_1/test.csv'
test_y_file_path = f'./F24_Proj3_data/split_1/test_y.csv'

train = pd.read_csv(train_file_path)
test = pd.read_csv(test_file_path)
test_y = pd.read_csv(test_y_file_path)

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
             'you', 'your', 'yours', 'their', 'they', 'his', 'her', 'she',
             'he', 'a', 'an', 'and', 'is', 'was', 'are', 'were', 'him',
             'himself', 'has', 'have', 'it', 'its', 'the', 'us']

We are doing some preprocessing:
- Replacing HTML tags from reviews with a space character
- Removing stop words
- Convert to lowercase
- remove rarely used words
Note: The token pattern below treats words separated by apostrophes as a single token rather two splitting it into two tokens.

CountVectorizer converts the dataset into a matrix of token counts, which is the bag of words model. Each row is a review, and each column is a unique token in the entire dataset. The value represents the token count in the document.

In [22]:
train['review'] = train['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)
test['review'] = test['review'].str.replace('&lt;.*?&gt;', ' ', regex=True)

vectorizer = CountVectorizer(
    preprocessor=lambda x: x.lower(),  # Convert to lowercase
    stop_words=stop_words,             # Remove stop words
    ngram_range=(1, 4),               # Use 1- to 4-grams
    min_df=0.001,                        # Minimum term frequency
    max_df=0.5,                       # Maximum document frequency
    token_pattern=r"\b[\w+\|']+\b" # Use word tokenizer
)

dtm_train = vectorizer.fit_transform(train['review'])

For sentiment analysis, we will use LIME. 

In [34]:
random.seed(42)

positive_reviews = test[test_y['sentiment'] == 1].sample(5, random_state=42)
negative_reviews = test[test_y['sentiment'] == 0].sample(5, random_state=42)
selected_reviews = pd.concat([positive_reviews, negative_reviews])

In [38]:
explainer = LimeTextExplainer(class_names=['Negative', 'Positive'])

def explain_review(review_text):
    #print(review_text)
    explanation = explainer.explain_instance(review_text, log_reg.predict_proba)
    
    explanation.show_in_notebook(text=True)

for i, row in selected_reviews.iterrows():
    explain_review(row['review'])


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got 1D array instead:
array=['Well, I fear that my review of this special won\'t heed much different observation than the others before me, but I literally just watched it- during a PBS membership drive- and frankly I\'m too excited NOT to say anything. To really appreciate the enigma that is Barbra Streisand, you have to look back before the movies. Before the Broadway phenomenon of the mid-60\'s. When television was still a young medium, there was a form of entertainment very prominent on the air that is but a memory today: musical variety. Some musical shows were weekly series, but others were single, one-time specials, usually showcasing the special talent of the individual performer. This is where we get the raw, uninhibited first looks at Streisand. She had already been a guest performer on other variety shows including Garry Moore, Ed Sullivan, and scored a major coup in a one-time only tandem appearance with the woman who would pass her the baton 

In [27]:
scaler = StandardScaler(with_mean=False)
X_train_scaled = scaler.fit_transform(dtm_train)

lasso_model = Lasso(alpha=0.00623)

lasso_model.fit(X_train_scaled, train['sentiment'])

# Get the coefficients
lasso_coefs = lasso_model.coef_
print('Lasso Coefficients:', lasso_coefs.sum())

# Create a DataFrame with feature names and their coefficients
feature_coef_df = pd.DataFrame(
    {'Feature': np.array(vectorizer.get_feature_names_out()),
     'Coefficient': lasso_coefs})

selected_features = feature_coef_df[feature_coef_df['Coefficient'] != 0]
vocabulary = list(selected_features['Feature'].values)

with open('./myvocab.txt', 'w') as file:
    # Iterate through the list and write each word to a new line
    for word in vocabulary:
        file.write(word + "\n")

len(selected_features)

Lasso Coefficients: -0.2245395020328098


1206

Setting a seed for consistency in random selection.

In [ ]:
random.seed(42)

In [ ]:
myvocab = open("myvocab.txt", "r").read().splitlines()
tfidf_vectorizer = TfidfVectorizer(vocabulary=myvocab, ngram_range=(1, 4))
dtm_test = tfidf_vectorizer.fit_transform(dtm_train['review'])
# log_reg.fit(dtm_test, test_y['sentiment'])
# print(myvocab)
# print(test_y)
# print(dtm_test)

# Need to set a seed before selecting samples

positive_reviews = test[test_y['sentiment'] == 1].sample(5, random_state=42)
negative_reviews = test[test_y['sentiment'] == 0].sample(5, random_state=42)
selected_reviews = pd.concat([positive_reviews, negative_reviews])

explainer = LimeTextExplainer(class_names=['Negative', 'Positive'])

def explain_review(review_text):
    print(review_text)
    explanation = explainer.explain_instance(review_text, log_reg.predict_proba, num_features=10)
    
    explanation.show_in_notebook(text=True)

for i, row in selected_reviews.iterrows():
    explain_review(row['review'])
    
